In [2]:
from gensim.models import FastText
from konlpy.tag import Hannanum
from tqdm import trange
import sentencepiece as spm
import eunjeon

In [3]:
import json
#미술관,박물관
with open('corpus/NM.json','r') as f:
    nm=json.load(f)

In [4]:
#식물, 꽃관련
with open('corpus/NP.json','r') as f:
    np=json.load(f)

In [5]:
#연극, 극장
with open('corpus/NT.json','r') as f:
    nt=json.load(f)

In [6]:
#술집
with open('corpus/NA.json','r') as f:
    na=json.load(f)

In [7]:
#카페
with open('corpus/NC.json','r') as f:
    nc=json.load(f)

In [8]:
#음식
with open('corpus/TF.json','r') as f:
    tf=json.load(f)

In [9]:
nm_text = []
b = [x for x in nm]
for i in b:
    nm_text.append(i['document']['content']['text'])

In [10]:
np_text = []
b = [x for x in np]
for i in b:
    np_text.append(i['document']['content']['text'])

In [11]:
nt_text = []
b = [x for x in nt]
for i in b:
    nt_text.append(i['document']['content']['text'])

In [12]:
na_text = []
b = [x for x in na]
for i in b:
    na_text.append(i['document']['content']['text'])

In [13]:
nc_text = []
b = [x for x in nc]
for i in b:
    nc_text.append(i['document']['content']['text'])

In [14]:
tf_text = []
b = [x for x in tf]
for i in b:
    tf_text.append(i['document']['content']['text'])

In [15]:
import pandas as pd
nm_df = pd.DataFrame(nm_text, columns=['text'])
nm_df['target']=0
np_df = pd.DataFrame(np_text, columns=['text'])
np_df['target']=1
nt_df = pd.DataFrame(nt_text, columns=['text'])
nt_df['target']=2
na_df = pd.DataFrame(na_text, columns=['text'])
na_df['target']=3
nc_df = pd.DataFrame(nc_text, columns=['text'])
nc_df['target']=4
tf_df = pd.DataFrame(tf_text, columns=['text'])
tf_df['target']=5

In [16]:
df=pd.concat([nm_df,np_df,nt_df,na_df,nc_df,tf_df])
df

,text,target
0,"[전시회] 비욘더로드: 과몰입형, 출구없는 전시회이머시브 전시로 소문이 자자했던 비...",0
1,부천 가볼만한 전시회 심곡천 네모 갤러리에 다녀왔어요부천 가볼만한 전시회심곡천 네모...,0
2,"관람객이면서 동시에 퍼포먼스가 되는아트랩대전 고동환 전시회고동환 전시회 6개의 벽,...",0
3,만화 x 영화 홍콩만화애니메이션전 스케치 at 동대문 DDP 서울전시회 추천현재 동...,0
4,미술전시회 김현식 작가 개인전개인전 ‘玄’ 포스팅을 할까 합니다~화장한 10월의 어...,0
...,...,...
3243,서울역 맛집 유즈라멘을 추천합니다!유즈라멘은 서울역점에서 시작하여 안국역에 분점을 ...,5
3244,2TV 저녁 생생정보 / 생생정보통맛집오늘 서울 건대 인천 맛집우리의 저녁을 행복하...,5
3245,"안녕하세요, 하루의 마실입니다.오늘의 포스팅은 대학로/혜화에 위치한 홍대 돈가스 맛...",5
3246,"유튜브에서 빙수야가 유명해지기 전, 친구가 빙수야를 알려주면서 한 번 가보고 싶다고...",5


In [17]:
#중복된 정보 제거.
df.drop_duplicates(subset=['text'],inplace=True)

print(df['text'].nunique())

24467


In [18]:
#빈칸인 행 제거.
print(df.isnull().sum())

df= df.dropna(how='any')

text      0
target    0
dtype: int64


In [19]:
#한글을 제외한 글자 제거
df['text']=df['text'].str.replace("[^가-힣 ]", "")
df[:10]

<ipython-input-19-044abb49bebb>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text']=df['text'].str.replace("[^가-힣 ]", "")


,text,target
0,전시회 비욘더로드 과몰입형 출구없는 전시회이머시브 전시로 소문이 자자했던 비욘더로드...,0
1,부천 가볼만한 전시회 심곡천 네모 갤러리에 다녀왔어요부천 가볼만한 전시회심곡천 네모...,0
2,관람객이면서 동시에 퍼포먼스가 되는아트랩대전 고동환 전시회고동환 전시회 개의 벽 개...,0
3,만화 영화 홍콩만화애니메이션전 스케치 동대문 서울전시회 추천현재 동대문 배움...,0
4,미술전시회 김현식 작가 개인전개인전 포스팅을 할까 합니다화장한 월의 어느날 아이들...,0
5,코로나사태가 발생한지 벌써 약 년째그동안 전시회 페스티벌 등이 엄청 그리웠었죠월 위...,0
6,필립 할스만 점핑 어게인 필립 할스만 점핑 어게인긍정적인 에너지와 도전필립 할스...,0
7,여주 그린베이지소목장개인전시회지난주 금요일에 여주에 일이 있어 갔다가 협회 선생님이...,0
8,캠핑박람회 캠피크 수원감성 캠핑용품 캠핑카 전시회 캠피크 수원인스타 친구소환 ...,0
9,대전 유성구 이벤트유성국화전시회 즐기기언제나처럼 빵빵한 선물을 가지고 진행하는 유...,0


In [20]:
#빈칸인 행 제거.
import numpy as np
df['text'].replace('',np.nan, inplace=True)
print(len(df))
print(df.isnull().sum())

24467
text      23
target     0
dtype: int64


In [21]:
#전체 수 확인
df= df.dropna(how='any')
print(len(df))

24444


In [21]:
#분석에 불필요한 데이터 지정
stopwords=['을','는','이가','께서','것','의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','에서']

In [22]:
#형태소 분석을 한 자료 정제과정
mecab = eunjeon.Mecab()
import re
text=[]
for sentence in df['text']:
    text.append([word for word in mecab.morphs(sentence)if word not in stopwords])

In [27]:
#형태소 분석을 한 자료 정제과정
mecab = eunjeon.Mecab()
import re
train_input=[]
for sentence in df['text']:
    train_input.append(sentence)

In [23]:
print(len(train_input))

24444


In [35]:
emb_num = 128

embedding = FastText(text, vector_size=emb_num, window=12, min_count=5, sg=1)
embedding.save("fasttext_morph.model")

In [22]:
model_morphs = FastText.load("fasttext_morph.model")
model_morphs.wv.most_similar("식물")

[('화분', 0.8299668431282043),
 ('초록', 0.742006242275238),
 ('푸릇푸릇', 0.7015954256057739),
 ('꽃나무', 0.68097323179245),
 ('아기자기', 0.675330400466919),
 ('꽃식물', 0.6512247920036316),
 ('화초', 0.6481088399887085),
 ('가꿔', 0.6465709209442139),
 ('가꿔진', 0.6442983150482178),
 ('푸르르', 0.6395153999328613)]

In [38]:
model_morphs.wv.most_similar("음식")

[('요리', 0.7588831186294556),
 ('요양식', 0.6422492861747742),
 ('돋울', 0.6318318247795105),
 ('음식집', 0.6318122744560242),
 ('식', 0.6308205723762512),
 ('정말', 0.6195213794708252),
 ('음식점', 0.6168780326843262),
 ('메뉴', 0.6128191947937012),
 ('나오', 0.6119093298912048),
 ('데', 0.6101112365722656)]

In [39]:
with open('review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['text']))

In [40]:
corpus = "review.txt"
prefix = "review"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

In [27]:
vocab_file = "review.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)
line = "안녕하세요 만나서 반갑습니다"
pieces = vocab.encode_as_pieces(line)
ids = vocab.encode_as_ids(line)
print(line)
print(pieces)
print(ids)

안녕하세요 만나서 반갑습니다
['▁안녕하세요', '▁만나서', '▁반갑', '습니다']
[2728, 6211, 6282, 29]


In [28]:
tokened_sp = []
#형태소 분석을 한 자료 정제과정
for sentence in df['text']:
    tokened_sp.append(vocab.encode_as_pieces(sentence ))

KeyboardInterrupt: 

In [44]:
mb_num = 128

embedding = FastText(tokened_sp, vector_size=emb_num, window=10, min_count=2, sg=1)
embedding.save("fasttext_sp.model")

In [29]:
model_sp = FastText.load("fasttext_sp.model")
model_sp.wv.most_similar("식물")

[('▁식물', 0.8120625615119934),
 ('▁화분', 0.7289825677871704),
 ('화분', 0.6880204677581787),
 ('반려', 0.6422896385192871),
 ('▁식물이', 0.6018108129501343),
 ('정화식물', 0.5776996612548828),
 ('▁반려', 0.5761829018592834),
 ('동물', 0.5665410757064819),
 ('초록', 0.5616799592971802),
 ('▁키우기', 0.56104576587677)]

In [23]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
device = torch.device('cpu')
# device = torch.device("cuda")

In [1]:
!pip install torch

  Using cached torch-1.10.0-cp38-cp38-win_amd64.whl (226.6 MB)


You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [24]:
class SentenceDataset(Dataset):
    def __init__(self, dataset, target, tokenizer, fasttextModel, max_len):
        self.sentences = []
        with trange(len(dataset)) as tr:
            for i in tr:
                sen = dataset[i]
                sen = tokenizer(sen) #토큰화
                if len(sen) < max_len:
                    sen = sen + (max_len-len(sen)) * [""]
                sen = sen[:max_len]
                sen = fasttextModel[sen] #임베딩
                self.sentences.append(sen)
        self.labels = [np.int32(i) for i in target]

    def __getitem__(self, i):
        return (self.sentences[i],self.labels[i])

    def __len__(self):
        return (len(self.labels))

In [28]:
import sklearn
from sklearn.model_selection import train_test_split
train_target=np.array(df['target'])
train_input, test_input, train_target, test_target= train_test_split(
train_input, train_target, test_size=0.1)

In [29]:
mecab = eunjeon.Mecab()
max_len = 32
morphs_train = SentenceDataset(train_input, train_target, mecab.morphs, model_morphs.wv, max_len)
morphs_test = SentenceDataset(test_input, test_target, mecab.morphs,model_morphs.wv, max_len)

100%|█████████████████████████████████████████████████████████████████████████████| 2445/2445 [00:18<00:00, 130.73it/s]


In [32]:
#레이어를 주어 학습히기키
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
#최적의 모델을 찾기 위한 모듈
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [35]:
import numpy as np

In [36]:
model= Sequential()
model.add(LSTM(128))
model.add(Dense(3,activation='softmax'))
es=EarlyStopping(monitor='val_loss', mode='min',verbose=1, patience=4)
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['acc'])
history= model.fit(morphs_train,train_target, epochs=20, callbacks=[es, mc], batch_size=20)

ValueError: Failed to find data adapter that can handle input: <class '__main__.SentenceDataset'>, <class 'numpy.ndarray'>

In [37]:
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(morphs_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(morphs_test, batch_size=batch_size, num_workers=5, shuffle=True)

In [38]:
print(train_dataloader)

In [39]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [40]:
class LSTM(nn.Module):
    def __init__(self, input_size, num_classes, hidden_size, num_layers = 1):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes

        # self.emb = nn.Embedding(num_embeddings = len(vocab), embedding_dim = 128, padding_idx=0)

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, batch_first = True)

        self.linear = nn.Linear(hidden_size, hidden_size//2)
        self.linear2 = nn.Linear(hidden_size//2, num_classes)
        self.relu = nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.2)
    
        self.fc = nn.Sequential(self.linear, self.dropout, self.relu, self.linear2, self.dropout)
        # self.fc = nn.Sequential(self.linear, self.dropout)

    def forward(self, x_input):
        # x_input = self.emb(x_input)
        lstm_out, (h,c) = self.lstm(x_input)
        output = self.fc(lstm_out[:,-1,])
        
        # hidden = torch.cat((h[-2,:,:], h[-1,:,:]), dim = 1)
        # output=self.fc(hidden)
        # output = self.relu(output)
        # output = self.linear2(output)
        return output

In [ ]:
emb_num = 128
lstm = LSTM(emb_num, 6, 128, 2).to(device)

optimizer = torch.optim.Adam(lstm.parameters(), lr = 0.0003)
criterion = nn.CrossEntropyLoss()
epochs = 100


with trange(epochs) as tr:
    for i in tr:
        itloss = 0
        trainacc = 0
        testacc = 0
        
        lstm.train()
        for batch_id, (input, label) in enumerate(train_dataloader):
            optimizer.zero_grad()
            input = input.to(device)
            label = label.long().to(device)
            out = lstm(input)
            loss = criterion(out, label)
            loss.backward()
            optimizer.step()
            itloss += loss.cpu().item()
            trainacc += calc_accuracy(out,label)


        lstm.eval()
        for batch_idt, (input, label) in enumerate(test_dataloader):
            input = input.to(device)
            label = label.long().to(device)
            out = lstm(input)
            testacc += calc_accuracy(out,label)

        tr.set_postfix(trainacc="{0:.3f}".format(trainacc/(batch_id+1)), loss="{0:.3f}".format(itloss/(batch_id+1)),  testacc="{0:.3f}".format(testacc/(batch_idt+1)))

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]